In [112]:
from IPython.core.display import HTML
css_file = 'style.css'
HTML(open(css_file, 'r').read())

In [113]:
from pandas import read_excel, merge

In [114]:
from numpy import arange

In [115]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [116]:
# Plotly requires pip install plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# K-means clustering

## The data

In this hypothetical example we have a set of 99 patients using a smart watch, healthcare data app.  While the app continuously collects data, it does require the user to actively sync the data, so as to comply with healthcare regulatiuons.<br/>
To remind them to do so, a campaign consisting of e-mails, short-message services, WhatsApp messages, pamphlets, telephone and long letters are send out during the year (six types of campaigns).  These campaigns are numbered, 1 through 32, and a campaign consists only of a single modality.<br/>
A record is kept of which campaign the patient responded to.  The goal is to cluster the patients, so as to see which campaigns they respond to and which not, so as to target future campaigns.  Thus can be achieved through k-means clustering.  This machine learning technique creates centroids for each cluster based on geometrical distance between an individual data point and the calculated centroid.<br>
The example works just as well for a hypothetical business with a marketing campaign and looking at the respons with respect to sales.

In [117]:
# Importing an Excel spreadsheet with two sheets as two DataFrames
df_campaign = read_excel("PatientResponse.xlsx", sheet_name = 0)

In [118]:
df_response = read_excel("PatientResponse.xlsx", sheet_name = 1)
# Adding a column of value 1 to act as a count for that instance
df_response["n"] = 1

In [119]:
#see the last 5 data
df_campaign.tail()

,CampaignID,Type,Month
27,28,Long letter,November
28,29,SMS,November
29,30,email,December
30,31,WhatsApp,December
31,32,Long letter,December


In [120]:
df_response.tail()

,Patient,CampaignID,n
319,99,31,1
320,16,32,1
321,29,32,1
322,46,32,1
323,99,32,1


In [121]:
# Merge on the CampaignID columns
df = merge(df_campaign, df_response, on = "CampaignID")

In [122]:
df.tail()

,CampaignID,Type,Month,Patient,n
319,31,WhatsApp,December,99,1
320,32,Long letter,December,16,1
321,32,Long letter,December,29,1
322,32,Long letter,December,46,1
323,32,Long letter,December,99,1


In [123]:
# Create a pivot table to count each of the 32 campaigns
table = df.pivot_table(index = ["Patient"], columns = ["CampaignID"], values = "n")

In [124]:
table.tail()

CampaignID,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
Patient,,,,,,,,,,,,,,,,,,,,,
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
97,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
98,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0


In [125]:
# Fill NA values with 0 and reset the index to CampaignID
table = table.fillna(0).reset_index()

In [126]:
table.tail()

CampaignID,Patient,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
91,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
92,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
93,97,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
94,98,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
95,99,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [127]:
# Extracting the columns (32 campaigns)
cols = table.columns[1:]

In [128]:
cols

Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
      dtype='object', name='CampaignID')

## K-mean clustering

In [129]:
cluster = KMeans(n_clusters = 5) # At least 7-times times cluster = patients

In [130]:
# Predict the cluster from first patient down all the rows
#adding the cluster to the table coumn
table["cluster"] = cluster.fit_predict(table[table.columns[2:]])

In [131]:
table.tail()

CampaignID,Patient,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,cluster
91,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4
92,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1
93,97,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
94,98,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4
95,99,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0


In [132]:
table[cols].head()

CampaignID,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [133]:
# Principal component separation to create a 2-dimensional picture
pca = PCA(n_components = 2)
table['x'] = pca.fit_transform(table[cols])[:,0]
table['y'] = pca.fit_transform(table[cols])[:,1]
table = table.reset_index()

In [134]:
table.tail()

CampaignID,index,Patient,1,2,3,4,5,6,7,8,...,26,27,28,29,30,31,32,cluster,x,y
91,91,95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4,-0.121409,-0.437478
92,92,96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.666059,-0.843051
93,93,97,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,-0.474608,-0.368972
94,94,98,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4,-0.485692,-0.393428
95,95,99,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0,-0.831590,-1.119408


In [135]:
patient_clusters = table[["Patient", "cluster", "x", "y"]]

In [136]:
patient_clusters.tail()

CampaignID,Patient,cluster,x,y
91,95,4,-0.121409,-0.437478
92,96,1,0.666059,-0.843051
93,97,3,-0.474608,-0.368972
94,98,4,-0.485692,-0.393428
95,99,0,-0.831590,-1.119408


In [137]:
final = merge(df_response, patient_clusters)
final = merge(df_campaign, final);

In [138]:
final.tail()

,CampaignID,Type,Month,Patient,n,cluster,x,y
319,31,WhatsApp,December,64,1,3,-0.105954,-0.267206
320,32,Long letter,December,16,1,4,-0.433448,-0.020029
321,32,Long letter,December,46,1,3,-0.436308,-0.522012
322,32,Long letter,December,99,1,0,-0.831590,-1.119408
323,32,Long letter,December,29,1,4,-0.449028,0.029626


In [139]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

trace0 = plotly.graph_objs.Scatter(x = patient_clusters[patient_clusters.cluster == 0]["x"],
                    y = patient_clusters[patient_clusters.cluster == 0]["y"],
                    name = "Cluster 1",
                    mode = "markers",
                    marker = dict(size = 10,
                                 color = "rgba(15, 152, 152, 0.5)",
                                 line = dict(width = 1, color = "rgb(0,0,0)")))
trace1 = plotly.graph_objs.Scatter(x = patient_clusters[patient_clusters.cluster == 1]["x"],
                    y = patient_clusters[patient_clusters.cluster == 1]["y"],
                    name = "Cluster 2",
                    mode = "markers",
                    marker = dict(size = 10,
                                 color = "rgba(180, 18, 180, 0.5)",
                                 line = dict(width = 1, color = "rgb(0,0,0)")))
trace2 = plotly.graph_objs.Scatter(x = patient_clusters[patient_clusters.cluster == 2]["x"],
                    y = patient_clusters[patient_clusters.cluster == 2]["y"],
                    name = "Cluster 3",
                    mode = "markers",
                    marker = dict(size = 10,
                                 color = "rgba(132, 132, 132, 0.8)",
                                 line = dict(width = 1, color = "rgb(0,0,0)")))
trace3 = plotly.graph_objs.Scatter(x = patient_clusters[patient_clusters.cluster == 3]["x"],
                    y = patient_clusters[patient_clusters.cluster == 3]["y"],
                    name = "Cluster 4",
                    mode = "markers",
                    marker = dict(size = 10,
                                 color = "rgba(122, 122, 12, 0.8)",
                                 line = dict(width = 1, color = "rgb(0,0,0)")))
trace4 = plotly.graph_objs.Scatter(x = patient_clusters[patient_clusters.cluster == 4]["x"],
                    y = patient_clusters[patient_clusters.cluster == 4]["y"],
                    name = "Cluster 5",
                    mode = "markers",
                    marker = dict(size = 10,
                                 color = "rgba(230, 20, 30, 0.5)",
                                 line = dict(width = 1, color = "rgb(0,0,0)")))

data = [trace0, trace1, trace2, trace3, trace4]
plotly.offline.iplot(data)

In [140]:
final.head()

,CampaignID,Type,Month,Patient,n,cluster,x,y
0,1,email,January,10,1,4,-0.282221,-0.622973
1,1,email,January,23,1,3,-0.496037,-0.715865
2,1,email,January,24,1,2,-1.073771,0.126451
3,1,email,January,40,1,4,-0.856150,-0.289071
4,1,email,January,55,1,2,-0.357379,0.375035


In [141]:
# e-mails, short-message services, WhatsApp messages, pamphlets, telephone and long letters
#checking what is in the 0th cluster
final["0"] = final.cluster == 0
final.groupby("0").Type.value_counts()

0      Type       
False  WhatsApp       85
       email          76
       SMS            52
       Telephone      35
       Long letter    30
       Pamphlet       12
True   WhatsApp       16
       Telephone       8
       SMS             5
       Pamphlet        3
       Long letter     2
Name: Type, dtype: int64

In [142]:
# Number of patients in this(0th) cluster
final[final.cluster == 0]["Patient"].count()

34

In [143]:
# List of patients
final[final.cluster == 0].head()

,CampaignID,Type,Month,Patient,n,cluster,x,y,0
21,3,SMS,February,48,1,0,-0.715483,-0.929131,True
24,3,SMS,February,75,1,0,-0.698282,-0.774082,True
25,3,SMS,February,79,1,0,0.416195,-0.812995,True
29,4,WhatsApp,February,48,1,0,-0.715483,-0.929131,True
33,4,WhatsApp,February,35,1,0,-0.350569,-1.060899,True


In [144]:
final.head()

,CampaignID,Type,Month,Patient,n,cluster,x,y,0
0,1,email,January,10,1,4,-0.282221,-0.622973,False
1,1,email,January,23,1,3,-0.496037,-0.715865,False
2,1,email,January,24,1,2,-1.073771,0.126451,False
3,1,email,January,40,1,4,-0.856150,-0.289071,False
4,1,email,January,55,1,2,-0.357379,0.375035,False
